In [14]:
import openai
import pandas as pd

In [15]:
df = pd.read_excel('drone_dataset.xlsx')

In [16]:
df

,ID,Drone_Type,Speed,Colors,Safety_Details
0,1,Toy,58 kmph,"Grey, Green","No propeller guards, High maneuverability"
1,2,Toy,90 kmph,"Yellow, Grey","Equipped with GPS, Stable flight"
2,3,Toy,44 kmph,"White, Black","Collision detection, Stable flight"
3,4,Photography,92 kmph,"Red, Grey","No propeller guards, High speed"
4,5,Professional,119 kmph,"Grey, White","Propeller guards, Collision avoidance"
...,...,...,...,...,...
95,96,Racing,106 kmph,"Blue, Red","No propeller guards, High speed"
96,97,Racing,101 kmph,"Grey, Blue","No propeller guards, High speed"
97,98,Surveillance,59 kmph,"Yellow, Red","No propeller guards, High speed"
98,99,Toy,27 kmph,"Grey, White","Equipped with GPS, Stable flight"


In [17]:
df['response_strings'] = df.apply(lambda row: f"""Speed: {row['Speed']}
Colors: {row['Colors']}
Safety_Details: {row['Safety_Details']}""", axis=1)

In [18]:
print(df.response_strings.values[1])

Speed: 90 kmph
Colors: Yellow, Grey
Safety_Details: Equipped with GPS, Stable flight


In [19]:
system_prompt = "You are an assistant, detect drones."

In [20]:
all_conversations = []

for idx, row in df.iterrows():
    all_conversations.append({"messages": [{"role": "system", "content": system_prompt},
                                           {"role": "user", "content": row["Drone_Type"]},
                                           {"role": "assistant", "content": row['response_strings']}]})

In [21]:
all_conversations[0]

{'messages': [{'role': 'system',
   'content': 'You are an assistant, detect drones.'},
  {'role': 'user', 'content': 'Toy'},
  {'role': 'assistant',
   'content': 'Speed: 58 kmph\nColors: Grey, Green\nSafety_Details: No propeller guards, High maneuverability'}]}

In [22]:
import json

with open('instances.jsonl', 'w') as f:
    for conversation in all_conversations:
        json.dump(conversation, f)
        f.write('\n')

In [24]:
def get_api_key(file_path):
    with open(file_path, 'r') as file:
        return file.read().strip()

api = get_api_key('config.py')

client = openai.OpenAI(api_key=api)

In [33]:
openai.api_key = API_KEY


with open('instances.jsonl', 'rb') as f:
    response = client.files.create(file=f, purpose='fine-tune')


NameError: name 'API_KEY' is not defined

In [ ]:
response

In [ ]:
file_id = 'file-jyafPeRicBQ9RasGDSV7BNrA'

In [ ]:
response = client.fine_tuning.jobs.create(
    training_file = file_id,
    model = 'gpt-3.5-turbo'
)

In [ ]:
response

In [ ]:
job_id = 'ftjob-CoTFCKSRxbnuvN60ky6Uup0L'

In [ ]:
client.fine_tuning.jobs.list(limit=1)

In [ ]:
client.fine_tuning.jobs.retrieve(job_id)

In [ ]:
model_id = 'ft:gpt-3.5-turbo-0125:personal::9fQQAfbM'

In [ ]:
completion = client.chat.completions.create(
    model = model_id,
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Racing"}
    ]
)

In [ ]:
print(completion.choices[0].message.content)